## <center><strong>Notebook<span style= 'color: #51FCC6'> Describtion

## <center><strong>Importing<span style= 'color: #48E0DC'> Packeges

In [12]:
import sys
import time
import inspect
import warnings
import requests
import itertools
import cloudscraper
import numpy                         as np
import pandas                        as pd
import seaborn                       as sns
import matplotlib.pyplot             as plt

from itertools                       import count
from bs4                             import BeautifulSoup
from selenium                        import webdriver
from selenium.webdriver.common.by    import By
from selenium.webdriver.common.keys  import Keys
# from fake_useragent                  import UserAgent
# from IPython.display                 import set_matplotlib_formats
from selenium.webdriver.support.ui   import WebDriverWait
from selenium.webdriver.support      import expected_conditions as EC

In [14]:
warnings.filterwarnings('ignore')

MAX_PAGES        :int   = 20
COLORS           :list  = ['#51fcc6', '#48e0dc', '#5cd3f7', '#4895e0', '#517afc']
NUMERICS         :list  = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
                            'uint16', 'uint32', 'uint64']

data_jobs_titles :list  = ['Data entry', 'Data engineer',
                            'Data scientist', 'Data analyst',
                            'Machine Learning']

## <center><strong>Setting up the<span style= 'color: #5CD3F7'> Web scrapers

In [15]:
def scrape_page(url: str) -> BeautifulSoup:

    scraper = cloudscraper.create_scraper(delay= 10,browser= {
        'browser': 'chrome',
        'platform': 'windows',
        'desktop': True,
        'mobile': False}) 
    
    content = scraper.get(url).text
    soup    = BeautifulSoup(content)
    
    return soup

In [16]:
# Upwork scraper
def upwork_scraper(job_title : str,
                    page      : int) -> list:

    url         : str  = f'https://www.upwork.com/search/profiles/?page={page}&q={job_title}'
    soup        : str  = scrape_page(url)
    loaded_jobs : list = soup.find_all('div', class_= 'up-card-section up-card-hover')

    return  loaded_jobs


# Guru scraper
def guru_scraper(job_title : str,
                    page      : int) -> list:

    url         : str  = f'https://www.guru.com/d/freelancers/skill/{job_title}/pg/{page}/'
    soup        : str  = scrape_page(url)
    loaded_jobs : list = soup.find_all('div', class_= 'record record--avatarCheck findGuruRecord')
    loaded_page : int  = int(soup.find('li', class_= 'active').find('a').text)
    
    if loaded_page != page:
        loaded_jobs = []

    return loaded_jobs

Now we will try to calculate how much did each scraper take to collect the data

In [17]:
%timeit upwork_scraper('Data analytics', 1)
%timeit guru_scraper('Data analytics', 1)

The slowest run took 8.41 times longer than the fastest. This could mean that an intermediate result is being cached.
2.16 s ± 2.29 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
The slowest run took 16.64 times longer than the fastest. This could mean that an intermediate result is being cached.
3.28 s ± 3.96 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
guru_scraper('Data analytics', 1)

[<div class="record record--avatarCheck findGuruRecord" data-gid="710722" data-pids="640874,1354018" data-sid="4092047">
 <div class="record__check">
 <input :value="710722" @click="updateSelectedRecords(710722)" aria-label="Select Flexsin Technologies" class="g-chkradio g-chkradio--checkbox" id="guru__710722" type="checkbox" v-model="selectedRecords"/>
 <label for="guru__710722"></label>
 </div>
 <div class="record__details">
 <div class="record__header">
 <div class="record__header__identity">
 <div class="module_avatar freelancerAvatar">
 <div class="avatar">
 <a href="/freelancers/flexsin-technologies" rel="noopener noreferrer" target="_blank" title="Flexsin Technologies">
 <img align="absmiddle" alt="Flexsin Technologies" src="https://res.cloudinary.com/gurucom/image/upload/w_96,h_96,f_auto,dpr_2/v1707983121/pimg/FreelancerFiles/0/710/710722/puqyxf9zzllkgpjmwcqk.png"/>
 </a>
 </div>
 <div class="avatarinfo">
 <h3 class="freelancerAvatar__screenName">
 <a href="/freelancers/flexsin

## <center><strong>Collecting the Websites <span style = 'color: #4895e0'> HTML</span>

In [5]:
def retrieve_name(var):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var]

In [7]:
soups : dict = {'jobs_soups'     : [],
                'job_title'      : [],
                'platform'       : []}

for scraper in [guru_scraper, upwork_scraper]:
    scraper_name      : str  = retrieve_name(scraper)[0].split('_')[0].title()
    print(f'\nStarted scraping {scraper_name} data.')
    
    for job_title in data_jobs_titles:

        concated_jobs : list = []

        for i in count(0):
            page      : int  = i + 1
            jobs      : list = scraper(job_title, page)

            if (len(jobs) == 0) or (page >= MAX_PAGES):
                break

            for job in jobs:
                concated_jobs.append(job)


        print(f'Finished loading {job_title} jobs data from {scraper_name}')
        
        soups['jobs_soups']     .append(concated_jobs)
        soups['job_title']      .append(job_title)
        soups['platform']       .append(scraper_name)


Started scraping Guru data.
Finished loading Data entry jobs data from Guru
Finished loading Data engineer jobs data from Guru
Finished loading Data scientist jobs data from Guru
Finished loading Data analyst jobs data from Guru
Finished loading Machine Learning jobs data from Guru

Started scraping Upwork data.
Finished loading Data entry jobs data from Upwork
Finished loading Data engineer jobs data from Upwork
Finished loading Data scientist jobs data from Upwork
Finished loading Data analyst jobs data from Upwork
Finished loading Machine Learning jobs data from Upwork


In [14]:
for i in range(len(soups['job_title'])):
    if soups['job_title'][i] == 'Machine Learning':
        soups['job_title'][i] = 'ML developer'

## <center><strong> Collecting <span style= 'color: #517afc'>Cards</span> Data

In [17]:
stacked_df = pd.DataFrame(soups)

guru_df   = stacked_df[stacked_df['platform'] == 'Guru'].reset_index()
upwork_df = stacked_df[stacked_df['platform'] == 'Upwork'].reset_index()

guru_df['describtion']     = None
guru_df['earnings_amount'] = None
guru_df['feedback']        = None
guru_df['name']            = None
guru_df['job_titles']      = None
guru_df['addresse']        = None
guru_df['hour_rate']       = None
guru_df['minimum_pay']     = None
guru_df['skills']          = None

upwork_df['describtion']          = None
upwork_df['earnings_amount']      = None
upwork_df['feedback']             = None
upwork_df['name']                 = None
upwork_df['job_titles']           = None
upwork_df['country']              = None
upwork_df['hour_rate']            = None
upwork_df['consultations_offers'] = None
upwork_df['skills']               = None

#### **Loading Guru data**

In [18]:
for i, jobs in enumerate(guru_df['jobs_soups']):
    
    describtions     : list = []
    earnings_amounts : list = []
    feedbacks        : list = []
    names            : list = []
    job_titles       : list = []
    addresses        : list = []
    hour_rates       : list = []
    minimum_pays     : list = []
    skills           : list = []

    for job in jobs:

        try:      describtions     .append(job.find('p', 'serviceListing__desc').text.strip())
        except:   describtions     .append(np.nan)

        # Earnings per year
        try:      earnings_amounts .append(job.find('span', 'earnings__amount').text.strip())
        except:   earnings_amounts .append(np.nan)

        # Feedback
        try:      feedbacks        .append(job.find('span', 'freelancerAvatar__feedback').text.strip())
        except:   feedbacks        .append(np.nan)

        # Name
        try:      names            .append(job.find('h3', 'freelancerAvatar__screenName').find('a').text.strip())
        except:   names            .append(np.nan)

        # Job title
        try:      job_titles       .append(job.find('h2', 'serviceListing__title serviceListing__title--dark').find('a').text.strip())
        except:   job_titles       .append(np.nan)

        # Addresse
        try:      addresses        .append(job.find('span', 'freelancerAvatar__location')['title'].strip())
        except:   addresses        .append(np.nan)
        
        # Hour rate
        try:      hour_rates       .append(job.find('p', 'serviceListing__rates').text.split('.')[0].strip())
        except:   hour_rates       .append(np.nan)
        
        # Minimum pays
        try:      minimum_pays     .append(job.find('p', 'serviceListing__rates').text.split('·')[-1].strip())
        except:   minimum_pays     .append(np.nan)

        # Skills
        try:
            skills_temp = job.find_all('a', 'skillsList__skill skillsList__skill--hasHover')
            skills                 .append([skill.text.strip() for skill in skills_temp])
            
        except:   skills           .append(np.nan)
        
    # Now we are going to convert list into tuples.
    guru_df['describtion']     [i] = tuple(describtions)
    guru_df['earnings_amount'] [i] = tuple(earnings_amounts)
    guru_df['feedback']        [i] = tuple(feedbacks)
    guru_df['name']            [i] = tuple(names)
    guru_df['job_titles']      [i] = tuple(job_titles)
    guru_df['addresse']        [i] = tuple(addresses)
    guru_df['hour_rate']       [i] = tuple(hour_rates)
    guru_df['minimum_pay']     [i] = tuple(minimum_pays)
    guru_df['skills']          [i] = tuple(skills)

#### **Loading Upwork data**

In [19]:
for i, jobs in enumerate(upwork_df['jobs_soups']):
    
    describtions         : list = []
    earnings_amounts     : list = []
    feedbacks            : list = []
    names                : list = []
    job_titles           : list = []
    countries            : list = []
    hour_rates           : list = []
    skills               : list = []
    consultations_offers : list = []

    for job in jobs:
        try:      describtions          .append(job.find('div', 'up-line-clamp-v2 clamped').text.strip())
        except:   describtions          .append(np.nan)

        # Earnings per year
        try:      earnings_amounts      .append(job.find('p', 'mb-0').find('strong').text.strip())
        except:   earnings_amounts      .append(np.nan)

        # Feedback
        try:      feedbacks             .append(job.find('span', 'up-job-success-text').text.split('%')[0].strip())
        except:   feedbacks             .append(np.nan)

        # Name
        try:      names                 .append(job.find('div', 'identity-name').text.strip())
        except:   names                 .append(np.nan)

        # Job title
        try:      job_titles            .append(job.find('p', 'my-0 freelancer-title').find('strong').text.strip())
        except:   job_titles            .append(np.nan)

        # Country
        try:      countries             .append(job.find('span', attrs= {'itemprop': 'country-name'}).text.strip())
        except:   countries             .append(np.nan)
        
        # Hour rate
        try:      hour_rates            .append(job.find('div', 'grid-col-1 grid-col-sm-1 justify-self-start nowrap').find('strong').text.strip())
        except:   hour_rates            .append(np.nan)
        
        # Offers consultations
        try:      consultations_offers  .append(job.find('div', 'up-skill-badge up-badge up-badge-highlight up-badge-rounded-inverse').text.strip())
        except:   consultations_offers  .append('Doesn\'t offer consultations')
        
        # Skills
        try:
            skills_temp = job.find_all('div', 'up-skill-badge')
            skills                 .append([skill.text.strip() for skill in skills_temp])
            
        except:   skills           .append(np.nan)

    # Now we are going to convert list into tuples.
    upwork_df['describtion']          [i] = tuple(describtions)
    upwork_df['earnings_amount']      [i] = tuple(earnings_amounts)
    upwork_df['feedback']             [i] = tuple(feedbacks)
    upwork_df['name']                 [i] = tuple(names)
    upwork_df['job_titles']           [i] = tuple(job_titles)
    upwork_df['country']              [i] = tuple(countries)
    upwork_df['hour_rate']            [i] = tuple(hour_rates)
    upwork_df['consultations_offers'] [i] = tuple(consultations_offers)
    upwork_df['skills']               [i] = tuple(skills)

## <center><strong><span style= 'color: #517afc'>Saving</span> the data

In [20]:
upwork_dfs: list = [] # We will group all rows as dfs to stack them later.

for _, row in upwork_df.iterrows():
    del _

    df = pd.DataFrame({
        'describtion'           : row['describtion'],
        'earnings_amount'       : row['earnings_amount'],
        'feedback'              : row['feedback'],
        'name'                  : row['name'],
        'job_titles'            : row['job_titles'],
        'country'               : row['country'],
        'hour_rate'             : row['hour_rate'],
        'consultations_offers'  : row['consultations_offers'],
        'skills'                : row['skills']})

    df['job_title']   = row['job_title']

    upwork_dfs.append(df)

upwork_df = pd.concat(upwork_dfs, ignore_index= True, sort= False)
upwork_df.to_parquet(r'data/upwork_freelancers.parquet')
upwork_df.to_csv(r'data/upwork_freelancers.csv')

In [21]:
guru_dfs: list = [] # We will group all rows as dfs to stack them later.

for _, row in guru_df.iterrows():
    del _

    df = pd.DataFrame({
        'describtion'       : row['describtion'],
        'earnings_amount'   : row['earnings_amount'],
        'feedback'          : row['feedback'],
        'name'              : row['name'],
        'job_titles'        : row['job_titles'],
        'addresse'          : row['addresse'],
        'hour_rate'         : row['hour_rate'],
        'minimum_pay'       : row['minimum_pay'],
        'skills'            : row['skills']})

    df['job_title']   = row['job_title']

    guru_dfs.append(df)

guru_df = pd.concat(guru_dfs, ignore_index= True, sort= False)
guru_df.to_parquet(r'data/guru_freelancers.parquet')
guru_df.to_csv(r'data/guru_freelancers.csv')

encrypt.encrypt_json_file('../credentials.json')